In [2]:
import numpy as np
from pint import UnitRegistry
import periodictable

## Design Goals

- Simple, algebraic build-up of samples from base components 
    - H2O + 0.5*D2O = mixture
- Ability to specify concentrations 

## Round 1:

In [196]:
import numpy as np
import copy
import numbers

class BaseMaterial(object):
    '''Base class for all materials
    
    This class defines all of the basic properties and methods to be shared across material objects
    
    '''
    def __init__(self,name,mass=None,volume=None,density=None):
        self.name    = name
        self.mass    = mass
        self.volume  = volume
        self.density = density
    
    def _has_density(self):
        return (self.density is not None)
    
    def _has_volume(self):
        return (self.volume is not None)
    
    def _has_mass(self):
        return (self.mass is not None)
    
    def __mul__(self,factor):
        if not isinstance(factor,numbers.Number):
            raise TypeError(f'Can only multiply Component by numerical scale factor, not {type(other)}')
        component = copy.deepcopy(self)
        component.volume *= factor
        component.mass *= factor
        return component
    
    def __rmul__(self,factor):
        return self.__mul__(factor)
            
    def _combine_volume(self,other):
        if self._has_volume() and other._has_volume():
            self.volume = self.volume + other.volume
        else:
            self.volume = None
        
    def _combine_mass(self,other):
        if self._has_mass() and other._has_mass():
            self.mass = self.mass + other.mass
        else:
            self.mass = None
        
    def _combine_density(self,other):
        if self._has_mass() and other._has_mass() and self._has_density() and other._has_density():
            self.density = (self.mass + other.mass)/(self.mass/self.density + other.mass/other.density)
        else:
            self.density = None
            
    def _combine_all_properties(self,other):
        self._combine_volume(other)
        self._combine_mass(other)
        self._combine_density(other)
        

In [207]:
import numpy as np
import copy
import numbers

class Component(BaseMaterial):
    def __str__(self):
        return f'<Component M={self.mass:3.2f}  V={self.mass:3.2f}  P={self.density:3.2f} >'
    
    def __repr__(self):
        return self.__str__()
    
    def __add__(self,other):
        if isinstance(other,Component):
            if self.name == other.name:
                component = copy.deepcopy(self)
                component._combine_all_properties(other)
                return component
                
            else:
                mixture = ComponentMixture(
                                 name = self.name + '-' + other.name,
                                 mass=self.mass,
                                 volume=self.volume,
                                 density=self.density,
                                 )
                mixture.components[self.name] = self
                mixture.components[other.name] = other
                mixture._combine_all_properties(other)
                return mixture
            
        if isinstance(other,ComponentMixture):
            return other.__add__(self)
        else:
            raise TypeError(f'Unsure how to combine Component {self.name} with {type(other)}')

In [208]:
import numpy as np
import copy
import numbers
        
class ComponentMixture(BaseMaterial):
    def __init__(self,name,mass=None,volume=None,density=None):
        super().__init__(name,mass,volume,density)
        self.components = {}
        
    def __mul__(self,factor):
        if not isinstance(factor,numbers.Number):
            raise TypeError(f'Can only multiply Component by numerical scale factor, not {type(other)}')
            
        mixture = copy.deepcopy(self)
        mixture.volume *= factor
        mixture.mass *= factor
        for name,component in mixture.components.items():
            component.by_volume()._scale_function(other)
        
        if mixture._scale_function == mixture._scale_volume:
        elif mixture._scale_function == mixture._scale_mass:
            for name,component in mixture.components.items():
                component.by_mass()._scale_function(other)
        else:
            raise ValueError('Scale function not recognized!')
            
        return mixture
    
    def __rmul__(self,other):
        return self.__mul__(other)
    
    def __getitem__(self,key):
        return self.components[key]
        
    def contains(self,component):
        if component not in self.components:
            return False
        else:
            return True
        
    def mass_fraction(self,component):
        if not self.contains(component):
            raise ValueError(f'The Component {component} isn\'t in the Mixture {self.name}')
        return self.components[component].mass/self.mass
    
    def volume_fraction(self,component):
        if not self.contains(component):
            raise ValueError(f'The Component {component} isn\'t in the Mixture {self.name}')
        return self.components[component].volume/self.volume
        
    def __add__(self,other):
        if isinstance(other,Component):
            mixture = copy.deepcopy(self)
            
            if self.contains(other.name):
                mixture.components[other.name] = mixture.components[other.name] + other
            else:
                mixture.components[other.name] = other
                
            mixture.name += ('-' + other.name)
            mixture._combine_all_properties(other)
            return mixture
            
                
        elif isinstance(other,Mixture):
            mixture = copy.deepcopy(self)
            mixture.name = mixture.name + '-' + other.name
            
            for name,component in other.components.items():
                if other.contains(other.name):
                    mixture.components[other.name] = mixture.components[other.name] + other
                else:
                    mixture.components[other.name] = other
            
            mixture._combine_all_properties(other)
            return mixture
        else:
            raise TypeError(f'Unsure how to combine Mixture with {type(other)}')
        

In [209]:
class Sample(ComponentMixture):
    '''Simple alias of Mixture'''
    pass

class Stock(ComponentMixture):
    '''Simple alias of Mixture'''
    pass

In [210]:
H2O = Component('H2O',mass=5,volume=5,density=1.0)
D2O = Component('D2O',mass=5.55,volume=5,density=1.11)

mix = D2O + 0.2*H2O


vol1: 5
vol2: 1.0
mass1: 5
mass2: 1.0


In [211]:
mix.mass_fraction('H2O')

0.15267175572519084

In [212]:
mix.components

{'D2O': <Component M=5.55  V=5.55  P=1.11 >,
 'H2O': <Component M=1.00  V=1.00  P=1.00 >}

In [213]:
mix2 = mix + 0.1*H2O

vol1: 5
vol2: 0.5
mass1: 5
mass2: 0.5


In [214]:
mix2.mass_fraction('H2O')

0.2127659574468085

In [215]:
mix.components

{'D2O': <Component M=5.55  V=5.55  P=1.11 >,
 'H2O': <Component M=1.00  V=1.00  P=1.00 >}

In [216]:
print(mix2['D2O'].mass)
print(mix2['D2O'].volume)
print(mix2['D2O'].density)

5.55
5
1.11


In [217]:
(0.1*(mix2).by_volume()).components['D2O']

vol1: 6.5
vol2: 0.65
mass1: 7.05
mass2: 0.7067366704901341
vol1: 5
vol2: 0.5
mass1: 5.55
mass2: 0.555
vol1: 1.5
vol2: 0.15000000000000002
mass1: 1.5
mass2: 0.15000000000000002


<Component M=0.56  V=0.56  P=1.11 >

In [218]:
mix3 = 0.1*mix2
print(mix3['D2O'].mass)
print(mix3['D2O'].volume)
print(mix3['D2O'].density)

vol1: 6.5
vol2: 0.65
mass1: 7.05
mass2: 0.7067366704901341
vol1: 5
vol2: 0.5
mass1: 5.55
mass2: 0.555
vol1: 1.5
vol2: 0.15000000000000002
mass1: 1.5
mass2: 0.15000000000000002
0.555
0.5
1.11


In [219]:
D2O*0.1

vol1: 5
vol2: 0.5
mass1: 5.55
mass2: 0.555


<Component M=0.56  V=0.56  P=1.11 >